In [7]:
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Load data
df = pd.read_parquet('snotel_data.parquet')
df = df[df.station_triplet == '642:WA:SNTL']
df

,date,snow_depth,B01,B02,B03,B04,B05,B06,B07,B08,B09,B10,is_snow,granule_id,station_triplet
2432,2013-04-14T18:57:52.232Z,119,5757,5671,5544,5451,5809,1693,1491,-9999,91,-966,0,G2246987419-LPCLOUD,642:WA:SNTL
2433,2013-04-30T18:57:49.955Z,105,3946,3779,3500,3290,3616,1335,1154,-9999,188,-1305,0,G2246948039-LPCLOUD,642:WA:SNTL
2434,2013-05-16T18:58:02.239Z,68,1114,1163,1473,1459,1748,180,119,-9999,16,295,0,G2244110989-LPCLOUD,642:WA:SNTL
2435,2013-04-23T18:51:38.326Z,111,6032,5995,5623,5493,5559,1518,1601,-9999,278,-1610,0,G2246987880-LPCLOUD,642:WA:SNTL
2436,2013-04-14T18:57:28.350Z,119,5742,5656,5525,5432,5793,1687,1485,-9999,91,-966,0,G2246987473-LPCLOUD,642:WA:SNTL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4261,2023-04-27T19:11:39.194Z,115,4614,3759,4114,3949,3924,4492,4564,4725,1885,77,1,G2671167077-LPCLOUD,642:WA:SNTL
4262,2023-04-17T19:11:39.236Z,127,9649,9165,9055,8645,8789,8746,8651,8740,7199,2986,0,G2659758140-LPCLOUD,642:WA:SNTL
4263,2023-04-27T19:11:24.764Z,115,4612,3755,4105,3943,3913,4496,4560,4726,1886,77,1,G2671159039-LPCLOUD,642:WA:SNTL
4264,2023-05-02T19:11:38.024Z,103,4456,3803,4112,4020,4033,4595,4539,4546,1147,16,1,G2676740664-LPCLOUD,642:WA:SNTL


In [8]:

input_columns = [f"B{i:02d}" for i in range(1, 11)]
input_data = df[input_columns].values

output_columns = ['snow_depth']
output_data = df[output_columns].values

# Scale data
X_scaler = MinMaxScaler()
X_scaled = X_scaler.fit_transform(input_data).astype(np.float32)
Y_scaler = MinMaxScaler()
Y_scaled = Y_scaler.fit_transform(output_data).astype(np.float32)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, Y_scaled, test_size=0.33, random_state=42)

# Convert to tensors
x_training_data = torch.tensor(X_train, requires_grad=False)
y_training_data = torch.tensor(y_train, requires_grad=False)

# Define model
model = torch.nn.Linear(10, 1, bias=False)

# Define loss function and optimizer
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

# Train model
for epoch in range(20):

    # Forward pass: Compute predicted y by passing 
    # x to the model
    pred_y = model(x_training_data)

    # Compute and print loss
    loss = criterion(pred_y, y_training_data)

    # Zero gradients, perform a backward pass, 
    # and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 0.2780698239803314
epoch 1, loss 0.13084863126277924
epoch 2, loss 0.09988834708929062
epoch 3, loss 0.09318830817937851
epoch 4, loss 0.09155483543872833
epoch 5, loss 0.0909835621714592
epoch 6, loss 0.09063894301652908
epoch 7, loss 0.0903465673327446
epoch 8, loss 0.09006991237401962
epoch 9, loss 0.08980124443769455
epoch 10, loss 0.08953885734081268
epoch 11, loss 0.08928228169679642
epoch 12, loss 0.08903129398822784
epoch 13, loss 0.08878578245639801
epoch 14, loss 0.08854559063911438
epoch 15, loss 0.088310606777668
epoch 16, loss 0.08808071166276932
epoch 17, loss 0.08785577863454819
epoch 18, loss 0.08763569593429565
epoch 19, loss 0.08742036670446396
